In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("OlympicGamesReportes").getOrCreate()

# Ruta al archivo Parquet de la capa "Gold"
df_presentacion = spark.read.parquet("../processed/olympic_data_cleaned.parquet")

df_presentacion.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- nombre_atleta: string (nullable = true)
 |-- genero: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- altura: double (nullable = true)
 |-- peso: double (nullable = true)
 |-- equipo: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- juegos: string (nullable = true)
 |-- anio: integer (nullable = true)
 |-- temporada: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- deporte: string (nullable = true)
 |-- evento: string (nullable = true)
 |-- medalla: string (nullable = true)



In [5]:
# REPORTE MEDALLAS POR PAIS SEGUN AÑO

from pyspark.sql.functions import col, count

# Medallas por país, por año y por medalla
medallas_por_pais_anio = df_presentacion.filter(col("medalla") != "Sin Medalla") \
                                        .groupBy("pais", "anio", "medalla") \
                                        .agg(count("*").alias("conteo_medallas")) \
                                        .orderBy(col("anio"), col("pais"))

# Guardado final
ruta_medallas_pais_anio = "../presentacion/medallas_por_pais_anio.parquet"

medallas_por_pais_anio.write.mode("overwrite").parquet(ruta_medallas_pais_anio)

print("Guardado OK.")

Guardado OK.


In [6]:
# REPORTE CONTEO MEDALLAS POR PAIS

from pyspark.sql.functions import col, count

# Medallas por país
ranking_paises = df_presentacion.filter(col("medalla") != "Sin Medalla") \
                                .groupBy("pais") \
                                .agg(count("*").alias("total_medallas")) \
                                .orderBy(col("total_medallas").desc())

# Guardado final
ruta_ranking_paises = "../presentacion/ranking_paises.parquet"

ranking_paises.write.mode("overwrite").parquet(ruta_ranking_paises)

print("Guardado OK.")

Guardado OK.


In [7]:
# REPORTE POR DISCIPLINA

from pyspark.sql.functions import col, count, when, avg, countDistinct, round

# Participación y promedios por deporte
analisis_deporte = df_presentacion.groupBy("deporte") \
                                  .agg(
                                      countDistinct("ID").alias("total_atletas"),
                                      round(avg("edad"), 2).alias("promedio_edad"),
                                      round(avg("altura"), 2).alias("promedio_altura"),
                                      round(avg("peso"), 2).alias("promedio_peso"),
                                      count(when(col("medalla") == "Gold", 1)).alias("medallas_oro"),
                                      count(when(col("medalla") == "Silver", 1)).alias("medallas_plata"),
                                      count(when(col("medalla") == "Bronze", 1)).alias("medallas_bronce")
                                  ) \
                                  .orderBy(col("medallas_oro").desc())

# Guardado final
ruta_analisis_deporte = "../presentacion/analisis_deporte.parquet"

analisis_deporte.write.mode("overwrite").parquet(ruta_analisis_deporte)

print("Guardado OK")

Guardado OK
